In [1]:
import syft as sy


/home/teo/anaconda3/envs/PySyTFF/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
domain = sy.login(email="info@openmined.org", password="changethis", port=8081)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into node... done!


In [4]:
domain.datasets

['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__final__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_id', '_is_protocol', '_object2proto', '_proto2object', 'address', 'class_name', 'get_protobuf_schema', 'icon', 'id', 'metadatas', 'post_init', 'pprint', 'repr_short', 'sign', 'signed_type']
<class 'syft.core.node.common.node_service.dataset_manager.dataset_manager_messages.GetDatasetsResponse'>
['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__final__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module_

Idx,Name,Description,Assets,Id
[0],MedNIST Data,"The MedNIST dataset was gathered from several sets from TCIA, the RSNA Bone Age Challenge, and the NIH Chest X-ray dataset. The dataset is kindly made available by Dr. Bradley J. Erickson M.D., Ph.D. (Department of Radiology, Mayo Clinic) under the Creative Commons CC BY-SA 4.0 license. Label Count: 6 Label Mapping: {""AbdomenCT"": 0, ""BreastMRI"": 1, ""CXR"": 2, ""ChestCT"": 3, ""Hand"": 4, ""HeadCT"": 5} Image Dimensions: (64, 64) Total Images: 58954","[""images""] -> [""labels""] ->",de6332f0-2c20-4904-9816-96b74b26d4ad
[1],MedNIST Data,"The MedNIST dataset was gathered from several sets from TCIA, the RSNA Bone Age Challenge, and the NIH Chest X-ray dataset. The dataset is kindly made available by Dr. Bradley J. Erickson M.D., Ph.D. (Department of Radiology, Mayo Clinic) under the Creative Commons CC BY-SA 4.0 license. Label Count: 6 Label Mapping: {""AbdomenCT"": 0, ""BreastMRI"": 1, ""CXR"": 2, ""ChestCT"": 3, ""Hand"": 4, ""HeadCT"": 5} Image Dimensions: (64, 64) Total Images: 58954","[""images""] -> [""labels""] ->",52500a1e-c532-4b1e-a5ec-59a00f00ab92
[2],MedNIST Data,"The MedNIST dataset was gathered from several sets from TCIA, the RSNA Bone Age Challenge, and the NIH Chest X-ray dataset. The dataset is kindly made available by Dr. Bradley J. Erickson M.D., Ph.D. (Department of Radiology, Mayo Clinic) under the Creative Commons CC BY-SA 4.0 license. Label Count: 6 Label Mapping: {""AbdomenCT"": 0, ""BreastMRI"": 1, ""CXR"": 2, ""ChestCT"": 3, ""Hand"": 4, ""HeadCT"": 5} Image Dimensions: (64, 64) Total Images: 58954","[""images""] -> [""labels""] ->",45cd0fd0-ea52-4426-b642-2319528ae91d


In [6]:
domain.datasets[1]["images"]

['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__final__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_id', '_is_protocol', '_object2proto', '_proto2object', 'address', 'class_name', 'get_protobuf_schema', 'icon', 'id', 'metadatas', 'post_init', 'pprint', 'repr_short', 'sign', 'signed_type']
<class 'syft.core.node.common.node_service.dataset_manager.dataset_manager_messages.GetDatasetsResponse'>
['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__final__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module_

<TensorPointer -> node:ca1ba0f4813f4f4e9b76a8ec6a0aceaa>

In [14]:
# download MedNIST.pkl
if not os.path.exists("./MedNIST.pkl"):
    os.system('curl -O "https://media.githubusercontent.com/media/shubham3121/datasets/main/MedNIST/MedNIST.pkl"')
else:
    print("MedNIST already downloaded")


MedNIST already downloaded


In [15]:
# Helper Methods

import os
import json
import pandas as pd
from PIL import Image
from enum import Enum
from collections import defaultdict
import numpy as np
from syft.core.adp.data_subject_list import DataSubjectList


def get_label_mapping():
    # the data uses the following mapping
    mapping = {
        "AbdomenCT": 0, 
        "BreastMRI": 1, 
        "CXR": 2, 
        "ChestCT": 3, 
        "Hand": 4, 
        "HeadCT": 5
    }
    return mapping

df = pd.read_pickle("./MedNIST.pkl")
mapping = get_label_mapping()

total_num = df.shape[0]
print("Columns:", df.columns)
print("Total Images:", total_num)
print("Label Mapping", mapping)

Columns: Index(['patient_id', 'image', 'label'], dtype='object')
Total Images: 58954
Label Mapping {'AbdomenCT': 0, 'BreastMRI': 1, 'CXR': 2, 'ChestCT': 3, 'Hand': 4, 'HeadCT': 5}


In [16]:
def get_data_description(data):
    unique_label_cnt = data.label.nunique()
    lable_mapping = json.dumps(get_label_mapping())
    image_size = data.iloc[0]["image"].shape
    description = f"The MedNIST dataset was gathered from several sets from TCIA, the RSNA Bone Age Challenge, and the NIH Chest X-ray dataset. The dataset is kindly made available by Dr. Bradley J. Erickson M.D., Ph.D. (Department of Radiology, Mayo Clinic) under the Creative Commons CC BY-SA 4.0 license.\n"
    description += f"Label Count: {unique_label_cnt}\n"
    description += f"Label Mapping: {lable_mapping}\n"
    description += f"Image Dimensions: {image_size}\n"
    description += f"Total Images: {data.shape[0]}\n"
    return description

In [17]:
data_subjects = DataSubjectList.from_series(df['patient_id'])

images = df['image']
images = np.dstack(images.values).astype(np.int32)
images = np.rollaxis(images,-1)
labels = df['label'].to_numpy().astype("int32")
image_data = sy.Tensor(images).private(min_val=0, max_val=255, data_subjects=data_subjects)
label_data = sy.Tensor(labels).private(min_val=0, max_val=5, data_subjects=data_subjects)



In [18]:
# creating/uploading the dataset

# Name of the dataset
name = f"MedNIST Data"
dataset_description = get_data_description(df)
# upload the MedNIST data
domain.load_dataset(
    assets={"images": image_data, "labels": label_data},
    name=name,
    description=dataset_description,
    use_blob_storage=True
)

Loading dataset... checking asset types...                              ['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__final__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_id', '_is_protocol', '_object2proto', '_proto2object', 'address', 'class_name', 'content', 'get_protobuf_schema', 'icon', 'id', 'post_init', 'pprint', 'repr_short', 'sign', 'signed_type']
<class 'syft.core.node.common.node_service.node_setup.node_setup_messages.GetSetUpResponse'>
Loading dataset... uploading...🚀                        ['Reply', 'Request', '_DomainMessageRegistry__domain_message_registry', '__annotations__', '__attr_allowlist__', '__class__', '__class_getitem

Uploading `images`: 100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  2.19it/s]


['Reply', 'Request', '_DomainMessageRegistry__domain_message_registry', '__annotations__', '__attr_allowlist__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__final__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__serde_overrides__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_id', '_is_protocol', '_object2proto', '_proto2object', 'address', 'check_permissions', 'class_name', 'get_permissions', 'get_protobuf_schema', 'get_registered_messages', 'icon', 'id', 'kwargs', 'payload', 'post_init', 'pprint', 'reply', 'reply_payload_type', 'reply_to', 'repr_short', 'request_payload_type', 'run', 'sign', 'signed_type']
<class 'syft.core.node.common.node_service.upload_service.upload_service_messages.UploadDataComp

Uploading `labels`: 100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 143.34it/s]

['Reply', 'Request', '_DomainMessageRegistry__domain_message_registry', '__annotations__', '__attr_allowlist__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__final__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__serde_overrides__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_id', '_is_protocol', '_object2proto', '_proto2object', 'address', 'check_permissions', 'class_name', 'get_permissions', 'get_protobuf_schema', 'get_registered_messages', 'icon', 'id', 'kwargs', 'payload', 'post_init', 'pprint', 'reply', 'reply_payload_type', 'reply_to', 'repr_short', 'request_payload_type', 'run', 'sign', 'signed_type']
<class 'syft.core.node.common.node_service.upload_service.upload_service_messages.UploadDataComp

In [19]:
domain.datasets[-1]["images"]

['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__final__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_id', '_is_protocol', '_object2proto', '_proto2object', 'address', 'class_name', 'get_protobuf_schema', 'icon', 'id', 'metadatas', 'post_init', 'pprint', 'repr_short', 'sign', 'signed_type']
<class 'syft.core.node.common.node_service.dataset_manager.dataset_manager_messages.GetDatasetsResponse'>
['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__final__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module_

<TensorPointer -> node:4c262662e8eb44fcaf389a847ddaa028>

In [11]:
domain.load_dataset(
    assets={"images": image_data, "labels": label_data},
    name=name,
    description=dataset_description,
    use_blob_storage=True
)

Loading dataset... checking asset types...                              ['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__final__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_id', '_is_protocol', '_object2proto', '_proto2object', 'address', 'class_name', 'content', 'get_protobuf_schema', 'icon', 'id', 'post_init', 'pprint', 'repr_short', 'sign', 'signed_type']
<class 'syft.core.node.common.node_service.node_setup.node_setup_messages.GetSetUpResponse'>
Loading dataset... uploading...🚀                        ['Reply', 'Request', '_DomainMessageRegistry__domain_message_registry', '__annotations__', '__attr_allowlist__', '__class__', '__class_getitem

Uploading `images`: 100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  2.24it/s]


['Reply', 'Request', '_DomainMessageRegistry__domain_message_registry', '__annotations__', '__attr_allowlist__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__final__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__serde_overrides__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_id', '_is_protocol', '_object2proto', '_proto2object', 'address', 'check_permissions', 'class_name', 'get_permissions', 'get_protobuf_schema', 'get_registered_messages', 'icon', 'id', 'kwargs', 'payload', 'post_init', 'pprint', 'reply', 'reply_payload_type', 'reply_to', 'repr_short', 'request_payload_type', 'run', 'sign', 'signed_type']
<class 'syft.core.node.common.node_service.upload_service.upload_service_messages.UploadDataComp

Uploading `labels`: 100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 182.81it/s]

['Reply', 'Request', '_DomainMessageRegistry__domain_message_registry', '__annotations__', '__attr_allowlist__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__final__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__serde_overrides__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_id', '_is_protocol', '_object2proto', '_proto2object', 'address', 'check_permissions', 'class_name', 'get_permissions', 'get_protobuf_schema', 'get_registered_messages', 'icon', 'id', 'kwargs', 'payload', 'post_init', 'pprint', 'reply', 'reply_payload_type', 'reply_to', 'repr_short', 'request_payload_type', 'run', 'sign', 'signed_type']
<class 'syft.core.node.common.node_service.upload_service.upload_service_messages.UploadDataComp

In [13]:
domain.datasets[1]["images"]

['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__final__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_id', '_is_protocol', '_object2proto', '_proto2object', 'address', 'class_name', 'get_protobuf_schema', 'icon', 'id', 'metadatas', 'post_init', 'pprint', 'repr_short', 'sign', 'signed_type']
<class 'syft.core.node.common.node_service.dataset_manager.dataset_manager_messages.GetDatasetsResponse'>
['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__final__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module_

<TensorPointer -> node:ca1ba0f4813f4f4e9b76a8ec6a0aceaa>